In [165]:
!python3 -m pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [166]:
from datasets import load_dataset

In [167]:
dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [168]:
from datasets import load_dataset, Dataset

# Load CSV file as a dataset
datasetnew = load_dataset('csv', data_files='stereotypes.csv')
datasetnew = datasetnew.remove_columns(['Unnamed: 0'])
datasetnew = datasetnew['train']

# Step 1: Rename columns
datasetnew = datasetnew.rename_column("Sentence", "Text")

datasetnew = Dataset.from_dict({
    "label": datasetnew["Label"],
    "text": datasetnew["Text"]
})

# Change labels to 1 if "yes" and 0 if "no"
def update_labels(example):
    example["label"] = 1 if example["label"].lower() == "stereotypical" else 0
    return example

# Apply the transformation
datasetnew = datasetnew.map(update_labels)

# Verify the result
print(datasetnew)

Map: 100%|██████████| 2200/2200 [00:00<00:00, 49393.88 examples/s]

Dataset({
    features: ['label', 'text'],
    num_rows: 2200
})


In [169]:
split_dataset = datasetnew.train_test_split(test_size=0.5)

In [170]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets_new = split_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 1100/1100 [00:00<00:00, 10173.12 examples/s]


In [171]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [172]:
train_dataset = tokenized_datasets_new["train"]
eval_dataset = tokenized_datasets_new['test']

In [173]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [174]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [175]:
!python3 -m pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [176]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [177]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions = predictions, references = labels)

In [178]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [179]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

[codecarbon WARNING @ 20:54:14] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 20:54:14] [setup] RAM Tracking...
[codecarbon INFO @ 20:54:14] [setup] GPU Tracking...
[codecarbon INFO @ 20:54:14] No GPU found.
[codecarbon INFO @ 20:54:14] [setup] CPU Tracking...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
[codecarbon WARNING @ 20:54:14] No CPU tracking mode foun

In [ ]:
trainer.train()

  0%|          | 2/414 [00:02<07:46,  1.13s/it][codecarbon INFO @ 20:54:21] Energy consumed for RAM : 0.000375 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 20:54:21] Energy consumed for all CPUs : 0.002657 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:54:21] 0.003032 kWh of electricity used since the beginning.
  1%|▏         | 6/414 [00:06<06:31,  1.04it/s][codecarbon INFO @ 20:54:25] Energy consumed for RAM : 0.000800 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 20:54:25] Energy consumed for all CPUs : 0.005668 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:54:25] 0.006468 kWh of electricity used since the beginning.
  4%|▎         | 15/414 [00:14<06:09,  1.08it/s][codecarbon INFO @ 20:54:33] Energy consumed for RAM : 0.000025 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 20:54:33] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:54:33] 0.000202 kWh of electricity used since the beginning.
  4%|▍         | 18/414 [00:17<06:06,  1.08it/s][co

{'eval_loss': 0.17079117894172668, 'eval_accuracy': 0.9545454545454546, 'eval_runtime': 37.9673, 'eval_samples_per_second': 28.972, 'eval_steps_per_second': 3.635, 'epoch': 1.0}


[codecarbon INFO @ 20:57:06] Energy consumed for RAM : 0.000650 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 20:57:06] Energy consumed for all CPUs : 0.004605 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:57:06] 0.005256 kWh of electricity used since the beginning.
 34%|███▍      | 142/414 [02:51<23:12,  5.12s/it][codecarbon INFO @ 20:57:10] Energy consumed for RAM : 0.001075 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 20:57:10] Energy consumed for all CPUs : 0.007616 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:57:10] 0.008691 kWh of electricity used since the beginning.
 36%|███▌      | 149/414 [02:59<06:25,  1.45s/it][codecarbon INFO @ 20:57:18] Energy consumed for RAM : 0.000300 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 20:57:18] Energy consumed for all CPUs : 0.002126 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:57:18] 0.002426 kWh of electricity used since the beginning.
 37%|███▋      | 152/414 [03:02<05:17,  1.21s/it][codecarbon INFO @ 20:57:21] Energy consumed f

{'eval_loss': 0.12906160950660706, 'eval_accuracy': 0.9645454545454546, 'eval_runtime': 84.1301, 'eval_samples_per_second': 13.075, 'eval_steps_per_second': 1.64, 'epoch': 2.0}


 67%|██████▋   | 278/414 [25:34<42:13, 18.63s/it][codecarbon INFO @ 21:19:54] Energy consumed for RAM : 0.002559 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 21:19:54] Energy consumed for all CPUs : 0.018128 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:19:54] 0.020687 kWh of electricity used since the beginning.
 68%|██████▊   | 281/414 [25:37<15:37,  7.05s/it][codecarbon INFO @ 21:19:56] Energy consumed for RAM : 0.002934 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 21:19:56] Energy consumed for all CPUs : 0.020785 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:19:56] 0.023719 kWh of electricity used since the beginning.
 69%|██████▉   | 286/414 [25:42<04:10,  1.96s/it][codecarbon INFO @ 21:20:01] Energy consumed for RAM : 0.003359 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 21:20:01] Energy consumed for all CPUs : 0.023796 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 21:20:01] 0.027155 kWh of electricity used since the beginning.
 71%|███████   | 294/414 [25:49<01:58,  1.01it

{'eval_loss': 0.16610248386859894, 'eval_accuracy': 0.9609090909090909, 'eval_runtime': 44.1499, 'eval_samples_per_second': 24.915, 'eval_steps_per_second': 3.126, 'epoch': 3.0}
{'train_runtime': 6734.42, 'train_samples_per_second': 0.49, 'train_steps_per_second': 0.061, 'train_loss': 0.20262593808381454, 'epoch': 3.0}


TrainOutput(global_step=414, training_loss=0.20262593808381454, metrics={'train_runtime': 6734.42, 'train_samples_per_second': 0.49, 'train_steps_per_second': 0.061, 'total_flos': 868266482688000.0, 'train_loss': 0.20262593808381454, 'epoch': 3.0})

[codecarbon INFO @ 22:46:43] Energy consumed for RAM : 0.011612 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 22:46:43] Energy consumed for all CPUs : 0.082253 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:46:43] 0.093865 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:46:47] Energy consumed for RAM : 0.012037 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 22:46:47] Energy consumed for all CPUs : 0.085264 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:46:47] 0.097301 kWh of electricity used since the beginning.
[codecarbon WARNING @ 22:49:02] Background scheduler didn't run for a long period (138s), results might be inaccurate
[codecarbon INFO @ 22:49:02] Energy consumed for RAM : 0.011844 kWh. RAM Power : 6.0 W
[codecarbon INFO @ 22:49:02] Energy consumed for all CPUs : 0.083894 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 22:49:02] 0.095738 kWh of electricity used since the beginning.
[codecarbon WARNING @ 22:49:06] Background scheduler didn't run for a long 